In [5]:
using System.Collections.Generic;
using System.Threading;
using System.Threading.Tasks;

interface ICommand
{
    void Execute();
}

public class TestCommand : ICommand
{
    private readonly int id;
    private int counter = 0;

    public TestCommand(int id)
    {
        this.id = id;
    }

    public void Execute()
    {
        Console.WriteLine($"Поток {id} вызов {++counter}");
    }
}



In [7]:

        List<ICommand> commands = new List<ICommand>();
        for (int i = 0; i < 5; i++)
        {
            commands.Add(new TestCommand(i + 1));
        }

        using (CancellationTokenSource cts = new CancellationTokenSource())
        {
            Task executionTask = ExecuteCommands(commands, cts.Token);

            //запускаем команды 3 раза
            for (int i = 0; i < 3; i++)
            {
                Console.WriteLine($"--- Execution round {i + 1} ---");
                foreach (var command in commands)
                {
                    command.Execute();
                }

                //симуляция задержки между выполнениями
                await Task.Delay(1000);
            }

            //симуляция прерывания
            Console.WriteLine("Hard stopping...");
            cts.Cancel();

            try
            {
                await executionTask;
            }
            catch (OperationCanceledException)
            {
                Console.WriteLine("Execution was hard stopped.");
            }
        }


    static async Task ExecuteCommands(List<ICommand> commands, CancellationToken token)
    {
        while (!token.IsCancellationRequested)
        {
            foreach (var command in commands)
            {
                command.Execute();
            }

           //симуляция задержки
            await Task.Delay(1000, token);
        }
    }



Поток 1 вызов 1
Поток 2 вызов 1
Поток 3 вызов 1
Поток 4 вызов 1
Поток 5 вызов 1
--- Execution round 1 ---
Поток 1 вызов 2
Поток 2 вызов 2
Поток 3 вызов 2
Поток 4 вызов 2
Поток 5 вызов 2
--- Execution round 2 ---
Поток 1 вызов 4
Поток 2 вызов 3
Поток 1 вызов 3
Поток 3 вызов 3
Поток 4 вызов 3
Поток 5 вызов 3
Поток 2 вызов 4
Поток 3 вызов 4
Поток 4 вызов 4
Поток 5 вызов 4
--- Execution round 3 ---
Поток 1 вызов 6
Поток 1 вызов 5
Поток 2 вызов 6
Поток 3 вызов 5
Поток 4 вызов 5
Поток 5 вызов 5
Поток 2 вызов 5
Поток 3 вызов 6
Поток 4 вызов 6
Поток 5 вызов 6
Hard stopping...
Поток 1 вызов 7
Поток 2 вызов 7
Поток 3 вызов 7
Поток 4 вызов 7
Поток 5 вызов 7
Execution was hard stopped.
